In [ ]:
# Install required packages
!pip install langchain_groq langchain_core langchain_community
!pip install pypdf chromadb sentence_transformers

# Imports
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os



In [ ]:
# Initialize the Groq Chat LLM
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="api_key_here",
        model_name="llama3-70b-8192"
    )
    return llm



In [ ]:
# Create and save Chroma vector DB
def create_vector_db():
    loader = DirectoryLoader("/content/data", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db



In [ ]:
# Setup the QA Chain with a custom prompt
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_template = """You are a compassionate mental health chatbot. Use the following context (if helpful) to support the user, but rely on your own empathy and knowledge when appropriate.

Context: {context}
User: {question}
Chatbot:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        chain_type_kwargs={'prompt': PROMPT}
    )
    return qa_chain

# Detect if the user input is casual (e.g. greeting or polite expression)
def is_general_chat(query):
    general_keywords = ["hi", "hello", "hey", "how are you", "good morning", "good evening", "thank you", "thanks", "yo", "greetings"]
    return any(keyword in query.lower() for keyword in general_keywords)



In [ ]:
# Main chatbot loop
def main():
    print("Initializing Chatbot.........")
    llm = initialize_llm()

    db_path = "/content/chroma_db"

    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        query = input("\nHuman: ")
        if query.lower() == "exit":
            print("Chatbot: Take care of yourself, Goodbye!")
            break

        if is_general_chat(query):
            response = llm.invoke(query)
            print(f"Chatbot: {response.content}")
        else:
            response = qa_chain.invoke({"query": query})
            print(f"Chatbot: {response['result']}")

# Run the chatbot
if __name__ == "__main__":
    main()


Initializing Chatbot.........


<ipython-input-9-2f423e3a505e>:12: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)


Chatbot: Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?
Chatbot: I'm so sorry to hear that you're reaching out about work-related stress. It takes a lot of courage to acknowledge when things are getting tough. Can you tell me more about what's been going on at work that's causing you stress? Is it a specific task, a person, or a general feeling?
Chatbot: I'm so sorry to hear that you're going through this. Having a toxic boss can be incredibly stressful and affect your well-being. It's not uncommon for people in positions of power to abuse their authority, and it's not your fault.

Can you tell me more about what's been happening with your boss? What specific behaviors or actions have made you feel uncomfortable or stressed?
